### Minimizing the ROF functional using the primal dual hybrid gradient method (PDHG).

In [ ]:
from skimage.io import imread
import numpy as np
import scipy.sparse as sparse
import matplotlib.pyplot as plt
import time


# PDHG for the ROF model
def rof(f0, lambda_, h, maxNumIter, stopEps):
    m, n = f0.shape
    u = f0.ravel().copy()

    tau = 0.125
    sigma = tau
    theta = 1.0

    grad_x = xDerivative(m, n, h)
    grad_y = yDerivative(m, n, h)

    div_x = - grad_x.T
    div_y = - grad_y.T
    p = np.zeros((2, n*m))
    u_new = u.copy()
    u_bar = u.copy()
    for i in range(maxNumIter):
        # proximal operator of H^*
        tmp = p + sigma*np.array([grad_x*u_bar, grad_y*u_bar])
        denom = np.maximum(1, np.sqrt(tmp[0, :] ** 2 + tmp[1, :] ** 2))
        p = np.divide(tmp, denom)

        # proximal operator of G
        u_new = (u + tau * (div_x*p[0, :] + div_y*p[1, :] +
                            f0.ravel() / lambda_)) / (1 + tau / lambda_)

        # acceleration step
        u_bar = u_new + theta * (u_new - u)

        # stopping criterion
        threshold = np.linalg.norm(u_new - u, np.inf)
        if (threshold < stopEps):
            break

        # u = u_new
        np.copyto(u, u_new)

        if np.mod(i+1, 10) == 0:
            print('%d steps. ||u^(n+1)-u^n|| = %f' % (i+1, threshold))

    f = u_new.reshape(m, n)
    return f


# computes matrix representation of the x-derivative
def xDerivative(num_rows, num_cols, h):
    d = sparse.csr_matrix(sparse.spdiags(
        np.array([[-1, 1]]).T*np.ones((1, num_cols)), np.array([0, 1]), num_cols, num_cols) / h)
    d[num_cols-1, num_cols-1] = 0
    dx = sparse.csr_matrix(sparse.kron(sparse.eye(num_rows), d))
    return dx


# computes matrix representation of the y-derivative
def yDerivative(num_rows, num_cols, h):
    d = sparse.csr_matrix(sparse.spdiags(
        np.array([[-1, 1]]).T*np.ones((1, num_rows)), np.array([0, 1]), num_rows, num_rows) / h)
    d[num_rows-1, num_rows-1] = 0
    dy = sparse.csr_matrix(sparse.kron(d, sparse.eye(num_cols)))
    return dy


In [ ]:
# Regularization parameter for ROF model.
lambda_ = 0.02
h = 1
# PDHG parameters
maxIter = 1000
stopEps = 0.0001

# Prepare a noisy input image
from skimage import color, data
f = color.rgb2gray(data.cat()/255)
np.random.seed(42)
f = np.minimum(np.maximum(f + 0.1 * (np.random.rand(*f.shape) - 0.5), 0), 1)

# Rudin-Osher-Fatemi-denoising
tic = time.time()
fDenoised = rof(f, lambda_, h, maxIter, stopEps)
toc = time.time()
print('Elapsed time is', toc-tic, ' seconds.')

# Show the result
plt.subplot(1, 2, 1)
plt.imshow(f, interpolation='nearest',
           cmap=plt.cm.get_cmap('gray'), vmin=0, vmax=1)
plt.axis('off')
plt.title('Input image')

plt.subplot(1, 2, 2)
plt.imshow(fDenoised, interpolation='nearest',
           cmap=plt.cm.get_cmap('gray'), vmin=0, vmax=1)
plt.axis('off')
plt.title('ROF result')
plt.show()
